### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

All CSV files must have the following columns
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.

# Importing data into Dataframe

In [1]:
import json 
import pandas as pd 
import numpy as np
# from pandas.io.json import json_normalize 

# Now we are going to open a json file using json package
# json.load() :: retrieve the file in a dict format

# Open the file
with open('usa.gov_click_data.json') as f:
    # Load file as json
    records = [json.loads(line) for line in f]

In [2]:
records[0]

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 'c': 'US',
 'nk': 1,
 'tz': 'America/New_York',
 'gr': 'MA',
 'g': 'A6qOVH',
 'h': 'wfLQtf',
 'l': 'orofrog',
 'al': 'en-US,en;q=0.8',
 'hh': '1.usa.gov',
 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991',
 't': 1331923247,
 'hc': 1331822918,
 'cy': 'Danvers',
 'll': [42.576698, -70.954903]}

In [3]:
df_json = pd.json_normalize(records)

In [4]:
column_list = ['a','tz','r','u','t','hc','cy','ll']

In [5]:
df_json.columns.difference(column_list)

Index(['_heartbeat_', 'al', 'c', 'g', 'gr', 'h', 'hh', 'kw', 'l', 'nk'], dtype='object')

In [6]:
df_json = df_json.drop(columns=df_json.columns.difference(column_list)).dropna(how='all')

In [7]:
df_json.head(3)

,a,tz,r,u,t,hc,cy,ll
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,America/New_York,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1.331923e+09,1.331823e+09,Danvers,"[42.576698, -70.954903]"
1,GoogleMaps/RochesterNY,America/Denver,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1.331923e+09,1.308262e+09,Provo,"[40.218102, -111.613297]"
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,America/New_York,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1.331923e+09,1.331920e+09,Washington,"[38.9007, -77.043098]"


# Cleaning data

## Extract operating system and web browser

In [8]:
df_json['a'].str.extract(r'\(([^)]*)').head(3)

,0
0,Windows NT 6.1; WOW64
1,NaN
2,Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW6...


In [9]:
df_json['operating_sys'] = df_json['a'].str.extract(r'\(([^)]*)')

In [10]:
df_json['a'] = df_json.a.str.split(' ',n=1).str[0]

In [11]:
df_json[['a','operating_sys']].head(3)

,a,operating_sys
0,Mozilla/5.0,Windows NT 6.1; WOW64
1,GoogleMaps/RochesterNY,NaN
2,Mozilla/4.0,Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW6...


## Null values

In [12]:
df_json.isna().sum()

a                  0
tz                 0
r                  0
u                  0
t                  0
hc                 0
cy               521
ll               521
operating_sys    192
dtype: int64

In [13]:
df_json['cy'].value_counts()

Provo            121
White Plains     104
Washington        98
Palo Alto         68
Shrewsbury        46
                ... 
New Iberia         1
Addlestone         1
New Brunswick      1
Managua            1
Murfreesboro       1
Name: cy, Length: 1014, dtype: int64

In [14]:
df_json.fillna({'cy':'No city','operating_sys':'Unknown OS'}, inplace=True)

In [15]:
df_json['ll'].tolist()[:9]

[[42.576698, -70.954903],
 [40.218102, -111.613297],
 [38.9007, -77.043098],
 [-23.549999, -46.616699],
 [42.286499, -71.714699],
 [42.286499, -71.714699],
 [51.116699, 15.2833],
 nan,
 nan]

In [16]:
df_json['ll'] = df_json['ll'].apply(lambda x: [0,0] if x is np.nan else x)## Extract operating system and web browser

## Splitting longitude and latitude

In [17]:
LongLat = pd.DataFrame(df_json['ll'].tolist()).set_index(df_json.index).rename(columns={0:'longitude', 1:'latitude'})

In [18]:
df_json.loc[:,['longitude', 'latitude']] = LongLat
df_json.drop(columns='ll',inplace=True)

In [19]:
columns_mapper = {'a':'web_browser', 'tz':'time_zone','r':'from_url','u':'to_url', 'cy':'city', 't':'time_in', 'hc':'time_out'}

In [20]:
df_json.rename(columns=columns_mapper, inplace=True)

In [21]:
df_json.head(5)

,web_browser,time_zone,from_url,to_url,time_in,time_out,city,operating_sys,longitude,latitude
0,Mozilla/5.0,America/New_York,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1.331923e+09,1.331823e+09,Danvers,Windows NT 6.1; WOW64,42.576698,-70.954903
1,GoogleMaps/RochesterNY,America/Denver,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1.331923e+09,1.308262e+09,Provo,Unknown OS,40.218102,-111.613297
2,Mozilla/4.0,America/New_York,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1.331923e+09,1.331920e+09,Washington,Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW6...,38.900700,-77.043098
3,Mozilla/5.0,America/Sao_Paulo,direct,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,Braz,Macintosh; Intel Mac OS X 10_6_8,-23.549999,-46.616699
4,Mozilla/5.0,America/New_York,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273672e+09,Shrewsbury,Windows NT 6.1; WOW64,42.286499,-71.714699


## Short form URL

In [22]:
df_json['from_url'].str.split('/').str[0:3]

0            [http:, , www.facebook.com]
1            [http:, , www.AwareMap.com]
2                        [http:, , t.co]
3                               [direct]
4       [http:, , www.shrewsbury-ma.gov]
                      ...               
3555                            [direct]
3556         [http:, , www.facebook.com]
3557         [http:, , www.AwareMap.com]
3558                            [direct]
3559                     [http:, , t.co]
Name: from_url, Length: 3440, dtype: object

In [23]:
def url_extract(x):
    
    y = x.split('/')
    
    if len(y) < 3 :
        return x
    elif len(y[2]) <= 6:
        return x
    else:
        return '/'.join(y[:3])

In [24]:
df_json['from_url'].apply(url_extract).unique()

array(['http://www.facebook.com', 'http://www.AwareMap.com',
       'http://t.co/03elZC4Q', 'direct', 'http://www.shrewsbury-ma.gov',
       'http://plus.url.google.com', 'http://t.co/o1Pd0WeV',
       'http://t.co/ND7SoPyo', 'http://forum2.hkgolden.com',
       'http://t.co/6Cx4ROLs', 'http://t.co/5xlp0B34',
       'http://t.co/1ghGl5Pl', 'http://mobile.twitter.com',
       'http://www.aashtojournal.org', 'http://m.facebook.com',
       'http://t.co/wgctD0rd', 'http://t.co/ovz0wT2O', 'http://lalaf.com',
       'http://t.co/ihUhQF1r', 'https://twitter.com',
       'http://t.co/J6qInqf7', 'http://stocktwits.com',
       'http://t.co/lY3ZPJaN', 'http://t.co/6xAUIwJK',
       'http://t.co/eWtdEjjO', 'http://t.co/301GoVqI',
       'http://t.co/UKGBxj29',
       'http://36ohk6dgmcd1n-c.c.yom.mail.yahoo.net',
       'http://t.co/4tq2Lnj8', 'http://www.srh.noaa.gov',
       'http://t.co/x3LYPXyR', 'http://t.co/h6PbevjL',
       'http://t.co/yZP1pbGs', 'http://t.co/1LpYtOpY',
       'http://t.

In [25]:
df_json[['from_url','to_url']] = df_json[['from_url','to_url']].applymap(url_extract)

In [26]:
df_json = df_json[['web_browser','operating_sys', 'from_url', 'to_url', 'city', 'longitude', 'latitude', 'time_zone', 'time_in', 'time_out']]

In [27]:
df_json

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone,time_in,time_out
0,Mozilla/5.0,Windows NT 6.1; WOW64,http://www.facebook.com,http://www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,1.331923e+09,1.331823e+09
1,GoogleMaps/RochesterNY,Unknown OS,http://www.AwareMap.com,http://www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,1.331923e+09,1.308262e+09
2,Mozilla/4.0,Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW6...,http://t.co/03elZC4Q,http://boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,1.331923e+09,1.331920e+09
3,Mozilla/5.0,Macintosh; Intel Mac OS X 10_6_8,direct,http://apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,1.331923e+09,1.331923e+09
4,Mozilla/5.0,Windows NT 6.1; WOW64,http://www.shrewsbury-ma.gov,http://www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,1.331923e+09,1.273672e+09
...,...,...,...,...,...,...,...,...,...,...
3555,Mozilla/4.0,Windows NT 6.1; MSIE 9.0; Windows NT 6.1,direct,http://www.fda.gov,Paramus,40.944500,-74.070000,America/New_York,1.331927e+09,1.301298e+09
3556,Mozilla/5.0,Windows NT 5.1,http://www.facebook.com,http://www.okc.gov,Oklahoma City,35.471500,-97.518997,America/Chicago,1.331927e+09,1.307530e+09
3557,GoogleMaps/RochesterNY,Unknown OS,http://www.AwareMap.com,http://www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,1.331927e+09,1.308262e+09
3558,GoogleProducer,Unknown OS,direct,http://www.ahrq.gov,Mountain View,37.419201,-122.057404,America/Los_Angeles,1.331927e+09,1.327529e+09


In [28]:
df_json.to_csv('web-api.csv')